In [8]:
import pandas as pd
import os
import glob
import nibabel as nib
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline

from nipype.interfaces.fsl import BET

skullstrip = BET()


In [22]:
def is_directory(directory):
    if not os.path.exists(directory):
        os.mkdir(directory)
    return os.path.abspath(directory)

def sort_PD(root_directory,csv,origin,dest,scale,copy=True):
    sorted_directory = is_directory(dest)
    PD_metadata = pd.read_csv(csv)
    PD_dir = os.path.join(root_directory,origin)
    print(PD_dir)
    print("Now sorting Parkinson's Disease images")
    for identifier in PD_metadata['imageUID']:
        key = 'I' + str(identifier)
        for file in glob.glob(os.path.join(PD_dir,'*'+key+'*'+'.gz')):
            
            image = nib.load(file)
            img  = image.get_data()
            header = image.header
            # image  = resize(img/img.max(), (256, 256,100),order=3)
            uid = int(file.split('.nii.gz')[0].split('_')[-1][1:])
            print(img.shape)
#             scale = (228,256,48)
            header['dim'][1] = scale[0]
            header['dim'][2] = scale[1]
            header['dim'][3] = scale[2]
            rs_image = resize(img,scale, order=3, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=True, anti_aliasing_sigma=None)
            
            new_img = nib.Nifti1Image(rs_image, image.affine, header)
            nib.save(new_img, (dest+"/"+ file.split('/')[-1]))
    print("Done extracting the required  images from " +origin+' to ' + dest)
    
    

In [23]:
root_directory = os.getcwd()
sort_PD(root_directory,csv = 'axial_metadata.csv',
        origin='PD_BET',
        dest='pd_axial_v2',
        scale=(228,256,48))

/home/nikunj/Documents/598/EECS598-Project/PD_BET
Now sorting Parkinson's Disease images
(512, 512, 80)
(256, 228, 54)
(228, 256, 54)
(228, 256, 48)
(256, 228, 48)
(228, 256, 48)
(228, 256, 48)
(228, 256, 54)


KeyboardInterrupt: 

In [24]:
resized = glob.glob('pd_axial_v2/**/*.nii.gz', recursive=True)

In [25]:
for file in resized:
            image = nib.load(file)
            img  = image.get_data()
            if(img.shape !=(228,256,48)):
                print(file)
            